<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Prompt_Engineering/Rough_script_of_LLM_with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LLM( Llama v2 ) with pdf context
In previous 2 notebooks we have individually loaded Llama 7b and FAISS vectorstore . In this we combine both for incontext Q and A

##Installing required libraries
1. transformers: Hugging face libraries for loading pretrained transformer checkpoints.
2. accelerate : Manages communications between CPU and GPU more efficiently.
3.datasets : For loading datasets from huggingface for future fine tuning.
4. bitsandbytes : Used for quantization of model to run on limited resources.
5. einops :Einstein-Inspired Notation for operations
6. wandb :weights and biases for visualizations
7. Langchain : High level library assisting in creation and deployment of LLM apps : https://python.langchain.com/docs/get_started
8. sentence_transformers : SentenceTransformers 🤗 is a Python framework for state-of-the-art sentence, text and image embeddings. : https://huggingface.co/sentence-transformers
9. faiss-cpu / faiss-gpu :Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. : https://python.langchain.com/docs/integrations/vectorstores/faiss
10. pypdf : pypdf into array of documents, where each document contains the page content and metadata with page number.: https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
11. tiktoken : Used for tokenwise splitting of string or steam . Tbn that tokenization takes place according to tokens not words.





In [2]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb
!pip install langchain
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install faiss-gpu
!pip install pypdf
!pip install  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch


#####pdftextloader : https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

In [3]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader



In [ ]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from IPython.display import Markdown
if torch.cuda.is_available():
    print('cuda')
    device = torch.device("cuda")  # If GPU is available, use it.
else:
    device = torch.device("cpu")   # If GPU is not available, use the CPU.



cuda


Observations :
1. Sometimes the 404 Error occurs where the link to the urls is not accessible even if url is correct . In such case restarting the runtime helped.
2. I observed that formating of pypdf was the best . but the rest of the loaders could also be used.
PDF loaders : https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

##Part 1 : Going Sequentially
#### Loading Model --> Loading Embeddings and Vector Store --> Loading Context --> Splitting ,embedding and saving context --> Passing instruction and finding nearest context --> Creating Pormpt --> Passing prompt to LLM.

In [19]:
urls="https://arxiv.org/pdf/1706.03762.pdf"
loader=PyPDFLoader(urls)
data=loader.load()

In [13]:
content=' '
for i in data : content+=i.page_content
content

' Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and co

In [17]:
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=50)

texts = text_splitter.create_documents([content])


In [21]:
for t in data:
  print(len(t.page_content),t.metadata)
  t.metadata['new']=55
for t in data: print(len(t.page_content),t.metadata)

2853 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 0}
4260 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 1}
1826 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 2}
2481 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 3}
3169 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 4}
3448 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 5}
3305 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 6}
3149 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 7}
2969 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 8}
3111 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 9}
3229 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 10}
3229 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 11}
812 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 12}
814 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 13}
817 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 14}
2853 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 0, 'new': 55}
4260 {'source': '/tmp/tmp7jw3njyg/tmp.pdf', 'page': 1, 'new': 55}
1826 {'source': '/tmp/tmp7jw3njyg/tmp.pdf

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = FAISS.from_documents(texts, embeddings)

In [ ]:
question="What are the parts of transformers?"
contexts=db.similarity_search(question,k=2)
print(contexts)

[Document(page_content='former follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected feed-forward network. We employ a residual connection [ 11] around each of\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\nLayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produce outputs of dimension dmodel = 512 .\nDecoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the tw

In [ ]:
context=' '
for i in contexts:context+=i.page_content
context

' former follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected feed-forward network. We employ a residual connection [ 11] around each of\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\nLayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produce outputs of dimension dmodel = 512 .\nDecoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two\nsub-layers in each 

Input prompt for Llama is of the form :
```
prompt="""<s> [INST]
<<SYS>>
System guiding message
<</SYS>>
[/INST]"""
```
Sources:
1. https://huggingface.co/blog/llama2
2. https://discuss.huggingface.co/t/llama-2-7b-hf-repeats-context-of-question-directly-from-input-prompt-cuts-off-with-newlines/48250/10  
If not done so will result in the language model misbehaving . Therefore prompt engineering is important.

In [ ]:
prompt=f"""<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n
Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n
<</SYS>>
Context:{context}
Qustion:{question}
 [/INST]
"""
print(prompt)

<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.

Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.

<</SYS>>
Context: former follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [ 11] around each of
the two sub-layers, followed by layer n

In [ ]:
from IPython.display import Markdown

In [ ]:
# prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?'
inputs = tokenizer(prompt, return_tensors="pt",padding =True).to(device)
generate_ids = model.generate(inputs.input_ids, max_length=5000,top_k=1,top_p=0.5,temperature=0)


OutOfMemoryError: ignored

Skipping the prompt from the result.

In [ ]:
p=generate_ids[0][inputs['input_ids'].shape[1]:]

Markdown for better outputs.
Results are quite astonishing for a 7b parameter model.

In [ ]:
Markdown(tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True))
# print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

The Transformer architecture consists of the following parts:

1. Encoder: The encoder is composed of a stack of N identical layers, each of which consists of two sub-layers: a multi-head self-attention mechanism and a position-wise feed-forward network. The output of each sub-layer is passed through a residual connection and layer normalization.
2. Decoder: The decoder is also composed of a stack of N identical layers, with each layer consisting of a multi-head self-attention mechanism and a position-wise feed-forward network. The decoder also inserts a third sub-layer that performs attention over the output of the encoder stack.
3. Attention: The attention function is used to compute the weighted sum of the input values, where the weights are computed based on the similarity between the query and key values. The attention function can be described as a mapping from queries and keys to output values.
4. Multi-Head Attention: Multi-head attention is a variation of the attention function that allows the model to attend to different aspects of the input simultaneously. It consists of multiple attention layers running in parallel, each with its own set of weights.
5. Positional Encoding: Positional encoding is a technique used to add positional information to the input values. It is typically done by adding a fixed vector to each input value, which encodes its position in the sequence.
6. Layer Normalization: Layer normalization is a technique used to normalize the activations of each layer in the model. It helps to reduce the impact of vanishing gradients during training.
7. Residual Connection: Residual connections are used to add the input of a layer to its output, allowing the model to learn more complex functions.
8. Embeddings: Embeddings are used to represent each input value in a higher-dimensional space. They are typically learned during training and are used to compute the attention weights.
9. Dot-Product Attention: Dot-product attention is a type of attention mechanism that computes the attention weights by taking the dot product of the query and key values and scaling the result by a scalar value.
10. Multi-Head Dot-Product Attention: Multi-head dot-product attention is a variation of the dot-product attention mechanism that allows the model to attend to different aspects of the input simultaneously. It consists of multiple attention layers running in parallel, each with its own set of weights.
11. Softmax: The softmax function is used to normalize the attention weights, ensuring that they sum to 1. It is typically used at the output of the attention mechanism.
12. Layer Normalization: Layer normalization is a technique used to normalize the activations of each layer in the model. It helps to reduce the impact of vanishing gradients during training.
13. Positional Encoding: Positional encoding is a technique used to add positional information to the input values. It is typically done by adding a fixed vector to each input value, which encodes its position in the sequence.
14. Multi-Head Attention: Multi-head attention is a variation of the attention mechanism that allows the model to attend to different aspects of the input simultaneously. It consists of multiple attention layers running in parallel, each with its own set of weights.
15. Attention Mask: The attention mask is a binary vector that indicates which positions in the input sequence are valid for attention. It is typically used to prevent the model from attending to padding tokens or positions outside of the valid range.

These are the main components of the Transformer architecture. The specific implementation details may vary depending on the specific model and task, but these components provide a general understanding of how the Transformer works.

##Part 2 : Creating Chain using Langchain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [ ]:
prompt="""<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n
Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n
<</SYS>>
Context:{context}
Qustion:{question}
 [/INST]
"""

In [ ]:
prompt_t=PromptTemplate(input_variables=['context','question'],template=prompt)
prompt_t

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='<s>[INST] <<SYS>>\nYou are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n\nAnswer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n\n<</SYS>>\nContext:{context}\nQustion:{question}\n [/INST]\n', template_format='f-string', validate_template=True)

In [ ]:
LLMChain?

In [ ]:
chain=LLMChain(llm=model
               ,prompt=prompt_t)

ValidationError: ignored

In [4]:
class Cust_Chain_obj():
  def __init__(self,model,FAISS_obj,sys_prompt='',embeddings = HuggingFaceEmbeddings(),text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)):
    self.FAISS_obj=FAISS_obj
    self.model=model
    self.sys_prompt=sys_prompt
    self.embedding_obj=embeddings
    self.text_splitter_obj=text_splitter
    self.doc=None

  def set_embedding_obj(self,new_embedding_obj):
    self.embedding_obj=new_embedding_obj
  def set_sys_prompt(self,new_sys_prompt):
    self.sys_prompt=f"<<SYS>>{new_sys_prompt}<</SYS>>"
  def set_new_model(self,new_model):
    self.model=new_model
  def populate_vector_store(self,document,embedding_obj=None,text_splitter=None):
    if embedding_obj is None:
      embedding_obj=self.embedding_obj
    if (text_splitter is None):
      text_splitter=self.text_splitter_obj
    docs=text



/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


NameError: ignored

237.6